In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib
params = {"text.usetex" : True,
          "font.family" : "serif",
          "font.serif" : ["Computer Modern Serif"],
         'font.size':12}
plt.rcParams.update(params)
import ipywidgets as widgets
import datetime
import pandas as pd
import os
import hypermri
import scipy
import math
from scipy.optimize import curve_fit
import scipy.io as sio
from matplotlib import cm
import seaborn as sns
import hypermri.utils.utils_general as utg
import hypermri.utils.utils_spectroscopy as uts
import hypermri.utils.utils_fitting as utf
import sys
# define paths:
sys.path.append('../')
import TEMPLATE
rpath,basepath,spath=TEMPLATE.import_all_packages()
# Autoreload extension so that you dont have to reload the kernel every time something is changed in the hypermri package
%load_ext autoreload
%autoreload 2

%matplotlib widget


# 1. GBM data

In [ ]:
publication_path = os.path.join(os.path.abspath(os.path.join(basepath, os.pardir)),r'Publication/Final/06_Figure/')

In [ ]:
gbm_path = os.path.join(os.path.abspath(os.path.join(basepath, os.pardir)),'MISSION-GliomaCambridge/Slicespec/')
dataframe_GBM = []
studyfolder_names = ['', '', '', '', '', '', '']
for study_name in studyfolder_names:
    temp_df=pd.read_excel(gbm_path+study_name+'_results_final.xlsx',index_col=0)
    dataframe_GBM.append(temp_df)
dataframe_GBM = pd.concat(dataframe_GBM, ignore_index=True)

# visually determined which slice has a tumor:
tumor_slices = [[1,3],
               [1,3],
               [1,2,3],
               [1,2,3],
               [2],
               [2,3],
               [2,3]]
# manually putting into bool
is_tumor_slice=[True,False,True,
               True,False,True,
               True,True,True,
               True,True,True,
               False,True,False,
               False,True,True,
               False,True,True]

       
    

dataframe_GBM['tumor slice']=is_tumor_slice

# removing patient 0 since there were a lot of eddy currents acc. to the publication
dataframe_GBM=dataframe_GBM.drop([0,1,2])
dataframe_GBM.reset_index(drop=True,inplace=True)


# exclude datapoint with insane AUCR error
excluded_due_to_error_index=np.where(dataframe_GBM['dAUCR']>1)

dataframe_GBM=dataframe_GBM.drop(index=excluded_due_to_error_index[0])

dataframe_GBM.round(1)

In [ ]:
def lin_fun(x,M,k):
    return x*k+M

x_ax=np.arange(0,6,1)
mean_per_patient=dataframe_GBM.groupby('ID')['T'].mean().reset_index()['T']
std_per_patient=dataframe_GBM.groupby('ID')['dT'].mean().reset_index()['dT']
      
coeff,pcov = curve_fit(lin_fun,x_ax,mean_per_patient)
residuals = mean_per_patient - lin_fun(x_ax, *coeff)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((mean_per_patient-np.mean(mean_per_patient))**2)
r_squared = 1 - (ss_res / ss_tot)
print(coeff,np.sqrt(np.diag(pcov)))
fig,ax=plt.subplots(1,figsize=(3.42,3.42),tight_layout=True)

ax.errorbar(x_ax,mean_per_patient,color='k',yerr=std_per_patient,capsize=5,fmt='o')
#sns.regplot(x=x_ax,y=mean_per_patient,ci=95,ax=ax,scatter=False)

ax.set_yticks([20,25,30,35,40,45])
#ax.set_ylim([20,45])
ax.set_xticks(range(6))
ax.set_xlabel('Patient')
ax.set_ylabel('Brain temperature [°C]')
#ax.set_title(r'$R^2$='+str(np.round(r_squared,3)))
plt.savefig(publication_path+'GBM_patient_temperatures.png',dpi=300)

In [ ]:
tumor_slice_df_gbm = dataframe_GBM[dataframe_GBM['tumor slice'] == True]
no_tumor_slice_df_gbm = dataframe_GBM[dataframe_GBM['tumor slice'] == False]

# 2. Healthy volunteer CSI data

In [ ]:
concentration ='5mM'

In [ ]:
df_full = []
basepath_csi=os.path.join(os.path.abspath(os.path.join(basepath, os.pardir)),'/HV-brain-CSI')
for studyfolder_num in [122,127,128,129,130,131,132,133,134]:
    
    studyfolder ='HV-'+str(studyfolder_num)
    savepath=os.path.join(basepath_csi,studyfolder)
    
    results_dict =utg.load_as_pkl(savepath,'HV-'+str(studyfolder_num)+'_results_dictionary_'+str(concentration)+'.pkl')
    results_df = pd.read_excel(savepath+'/HV-'+str(studyfolder_num)+'_temp_freq_results_'+str(concentration)+'_processed.xlsx')
    results_df.insert(0, 'ID', results_dict['patient_info']['ID']) 
    # all patients are female apparently
    #results_df.insert(1, 'sex', results_dict['patient_info']['sex']) 
    results_df.insert(1, 'pyr_vol', results_dict['patient_info']['pyr_vol']) 
    results_df.insert(2, 'weight', results_dict['patient_info']['weight']) 
    results_df.insert(3, 'calibrationfunction', results_dict['calibrationfunction']) 
    
    df_full.append(results_df)
df_full=pd.concat(df_full)
df_full=df_full.reset_index(drop=True)
df_full=df_full.drop(['Unnamed: 0'],axis=1)
new_index = []
for n in range(9):
    for m in range(5):
        new_index.append(n)
df_full.insert(0, 'Patnum', new_index) 



In [ ]:
selected_columns = df_full[['Patnum', 'Tmean','dT','pixels']]
selected_columns.round(1).iloc[40:]

In [ ]:
plt.close('all')

x_ax = (df_full['Patnum'].unique())

mean_temps = []
std_temps= []
for pat in x_ax:
    pat_indices = np.squeeze(np.array(np.where(df_full.Patnum==pat),dtype=int))
    Temp_per_slice = df_full.iloc[pat_indices]['Tmean']
    std_per_slice = df_full.iloc[pat_indices]['dT']
    mean_temp_per_pat = np.nanmean(Temp_per_slice)
    std_temp_per_pat = np.nanmean(std_per_slice)
    mean_temps.append(mean_temp_per_pat)
    std_temps.append(std_temp_per_pat)
mean_temp=np.nanmean(mean_temps)
std_temp=np.nanmean(std_temps)

fig,ax=plt.subplots(1,figsize=(3.42,3.42),tight_layout=True)
healthy_pat_slices_csi=[]
healthy_pat_mean_csi=[]
for pat in x_ax:
    pat_indices = np.squeeze(np.array(np.where(df_full.Patnum==pat),dtype=int))
    mean_per_pat = np.nanmean(df_full.iloc[pat_indices]['Tmean'])
    healthy_pat_slices_csi.append(df_full.iloc[pat_indices]['Tmean'].values)
    healthy_pat_mean_csi.append(np.nanmean(df_full.iloc[pat_indices]['Tmean']))
    std_per_pat = np.nanmean(df_full.iloc[pat_indices]['dT'])
    ax.errorbar(pat,mean_per_pat,yerr=std_per_pat,fmt='-o',markersize=7,capsize=10,color='k',alpha=0.8)
    
import seaborn as sns
def lin_fun(x,M,k):
    return x*k+M
coeff,pcov = curve_fit(lin_fun,x_ax,mean_temps)

residuals = mean_temps - lin_fun(x_ax, *coeff)
ss_res = np.sum(residuals**2)

ss_tot = np.sum((mean_temps-np.mean(mean_temps))**2)

r_squared = 1 - (ss_res / ss_tot)

print(coeff,np.sqrt(np.diag(pcov)))
ax.set_yticks([20,25,30,35,40,45])
#ax.set_ylim([20,45])
ax.set_xticks(range(9))
ax.set_xlabel('Patient')
ax.set_ylabel('Brain temperature [°C]')
plt.savefig(publication_path+'CSI_healthy_patient_temperatures.png',dpi=300)

# 3. Healthy volunteer slicespec data

In [ ]:
basepath_utsw=os.path.join(os.path.abspath(os.path.join(basepath, os.pardir)),'SP_brain/Results')

dataframe_utsw = pd.read_excel(basepath_utsw+'/SP_healthy_volunteers_final.xlsx')
mean_per_patient_utsw=[]

for id_value in dataframe_utsw['ScanNum'].unique():
    subset = dataframe_utsw[dataframe_utsw['ScanNum'] == id_value]
    mean_per_patient_utsw.append(np.mean(subset['T']))

In [ ]:
dataframe_utsw.round(1).iloc[24:]

In [ ]:
plt.close('all')

x_ax = (dataframe_utsw['ScanNum'].unique())

mean_temps = []
std_temps= []
for pat in x_ax:
    pat_indices = np.squeeze(np.array(np.where(dataframe_utsw.ScanNum==pat),dtype=int))
    Temp_per_slice = dataframe_utsw.iloc[pat_indices]['T']
    std_per_slice = dataframe_utsw.iloc[pat_indices]['dT']
    mean_temp_per_pat = np.nanmean(Temp_per_slice)
    std_temp_per_pat = np.nanmean(std_per_slice)
    mean_temps.append(mean_temp_per_pat)
    std_temps.append(std_temp_per_pat)
mean_temp=np.nanmean(mean_temps)
std_temp=np.nanmean(std_temps)

fig,ax=plt.subplots(1,figsize=(3.42,3.42),tight_layout=True)
healthy_pat_slices_utsw=[]
healthy_pat_mean_utsw=[]
for pat in x_ax:
    pat_indices = np.squeeze(np.array(np.where(dataframe_utsw.ScanNum==pat),dtype=int))
    mean_per_pat = np.nanmean(dataframe_utsw.iloc[pat_indices]['T'])
    healthy_pat_slices_utsw.append(dataframe_utsw.iloc[pat_indices]['T'].values)
    healthy_pat_mean_utsw.append(np.nanmean(dataframe_utsw.iloc[pat_indices]['T']))
    std_per_pat = np.nanmean(dataframe_utsw.iloc[pat_indices]['dT'])
    ax.errorbar(pat,mean_per_pat,yerr=std_per_pat,fmt='-o',markersize=7,capsize=10,color='k',alpha=0.8)
    
import seaborn as sns
def lin_fun(x,M,k):
    return x*k+M
coeff,pcov = curve_fit(lin_fun,x_ax,mean_temps)

residuals = mean_temps - lin_fun(x_ax, *coeff)
ss_res = np.sum(residuals**2)

ss_tot = np.sum((mean_temps-np.mean(mean_temps))**2)

r_squared = 1 - (ss_res / ss_tot)


#sns.regplot(x=x_ax,y=mean_temps,ci=95,ax=ax,scatter=False)
#ax.hlines(mean_temp,0,8,linestyle='dashed',color='C3',label='Mean')
print(coeff,np.sqrt(np.diag(pcov)))
#ax.hlines(38.5,0,8,linestyle='dashed',color='C0',label='Literature value')

#ax.hlines(mean_temp+std_temp,0,8,color='C2')
#ax.hlines(mean_temp-std_temp,0,8,color='C2')

ax.set_yticks([25,30,35,40])
#ax.set_ylim([20,45])
ax.set_xticks(range(7))
ax.set_xlabel('Dataset')
ax.set_ylabel('Brain temperature [°C]')
plt.savefig(publication_path+'SP_healthy_patient_temperatures.png',dpi=300)

In [ ]:
healthy_brain_slicespec_all_temp_values=np.load(basepath_utsw+'/human_slicespec_healthy_brain_temp_values.npy')

# 4. Kidney data

In [ ]:
def load_dataframe(studyfolder,exam_num):
    renalpath = os.path.join(os.path.abspath(os.path.join(basepath, os.pardir)),'RenalData/Results/')
    path = renalpath+'MRE-'+studyfolder+'_'+str(exam_num)+'_results.xlsx'
    df = pd.read_excel(path)
    return df

In [ ]:
studyfolder_nums = ['','','','','','','','','','','','']
dataframe_Renal=pd.DataFrame()
c=0
for studyfolder in studyfolder_nums:
    if studyfolder in ['','']:
        for exam_num in range(1,3):
            df = load_dataframe(studyfolder,exam_num)
            df['Unnamed: 0'] = np.ones(((df['slice'].max()+1)*(df['channel'].max()+1),1),dtype=int)*c
            df = df.rename(columns={'Unnamed: 0':'ScanNum'})
            c+=1
            dataframe_Renal = pd.concat([dataframe_Renal,df])
    else:
        df = load_dataframe(studyfolder,exam_num=1)
        df['Unnamed: 0'] = np.ones(((df['slice'].max()+1)*(df['channel'].max()+1),1),dtype=int)*c
        df = df.rename(columns={'Unnamed: 0':'ScanNum'})
        c+=1
        dataframe_Renal = pd.concat([dataframe_Renal,df])


In [ ]:
dataframe_Renal['ID','slice']

In [ ]:
plt.close('all')

x_ax = (dataframe_Renal['ScanNum'].unique())

mean_temps = []
std_temps= []
num_values=[]
error_of_mean_renal=[]
for pat in x_ax:
    pat_indices = np.squeeze(np.array(np.where(dataframe_Renal.ScanNum==pat),dtype=int))
    Temp_per_slice = dataframe_Renal.iloc[pat_indices]['T']
    num_value=dataframe_Renal.iloc[pat_indices]['nT']
    std_per_slice = dataframe_Renal.iloc[pat_indices]['dT']
    mean_temp_per_pat = np.nanmean(Temp_per_slice)
    std_temp_per_pat = np.nanmean(std_per_slice)
    num_values.append(np.nansum(num_value))
    mean_temps.append(mean_temp_per_pat)
    std_temps.append(std_temp_per_pat)

    lst=dataframe_Renal.iloc[pat_indices]['dT']
    foo = [x for x in lst if x != 0]

mean_temp=np.nanmean(mean_temps)
std_temp=np.nanmean(std_temps)
non_nan_ids= x_ax[np.where(~np.isnan(mean_temps))[0]]
mean_temps=np.array(mean_temps)
std_temps=np.array(std_temps)
mean_temps_nonnan_renal = mean_temps[~np.isnan(mean_temps)]
std_temps_nonnan_renal = std_temps[~np.isnan(std_temps)]

x_ax_non_nan_renal=np.array(range(0,non_nan_ids.shape[0]))
fig,ax=plt.subplots(1,figsize=(3.42,3.42),tight_layout=True)
ax.errorbar(x_ax_non_nan_renal,mean_temps_nonnan_renal,yerr=std_temps_nonnan_renal,fmt='o',markersize=7,capsize=10,color='k',alpha=0.8)
ax.set_yticks([25,30,35,40])
#ax.set_ylim([20,45])
ax.set_xticks(range(non_nan_ids.shape[0]))
ax.set_xlabel('Dataset')
ax.set_ylabel('Kidney temperature [°C]')



# Compute error of the mean

In [ ]:
fig,ax=plt.subplots(1,figsize=(3.42,3.42),tight_layout=True)

gbm_x=np.arange(0,6,1)
gbm_std_t=[]
gbm_mean_t=[]
for n in dataframe_GBM['ID'].unique():
    idx=np.squeeze(np.array(np.where(dataframe_GBM.ID==n),dtype=int))
    stds=dataframe_GBM.iloc[idx]['dT']
    try:
        std_of_mean=np.sqrt(np.sum(stds**2)) / len(stds)
    except:
        std_of_mean=stds
    mean=np.nanmean(dataframe_GBM.iloc[idx]['T'])
    gbm_std_t.append(std_of_mean)
    gbm_mean_t.append(mean)
    
ax.errorbar(gbm_x,gbm_mean_t,color='r',yerr=gbm_std_t,capsize=5,fmt='o',markersize=5,label='GBM')
print('GBM, STD per slice',gbm_std_t)

x_ax_csi = (df_full['Patnum'].unique())
for pat in x_ax_csi:
    pat_indices = np.squeeze(np.array(np.where(df_full.Patnum==pat),dtype=int))
    mean_per_pat = np.nanmean(df_full.iloc[pat_indices]['Tmean'])
    std_of_mean=np.sqrt(np.sum(df_full.iloc[pat_indices]['dT']**2)) / len(df_full.iloc[pat_indices]['dT'])
    ax.errorbar(pat+6,mean_per_pat,yerr=std_of_mean,fmt='-o',markersize=5,capsize=5,color='k',label='Healthy brain (CSI)')

    print('Healthy CSI, STD per slice',std_of_mean)
    
    
x_ax_utsw = (dataframe_utsw['ScanNum'].unique())
for pat in x_ax_utsw:
    pat_indices = np.squeeze(np.array(np.where(dataframe_utsw.ScanNum==pat),dtype=int))
    mean_per_pat = np.nanmean(dataframe_utsw.iloc[pat_indices]['T'])
    std_per_pat = np.sqrt(np.sum(dataframe_utsw.iloc[pat_indices]['dT']**2)) / len(dataframe_utsw.iloc[pat_indices]['dT'])
    ax.errorbar(pat+16,mean_per_pat,yerr=std_per_pat,fmt='-o',markersize=5,capsize=5,color='C0',label='Healthy brain (slicespec)')

    print('Healthy SP, STD per slice',std_per_pat)
    
ax.errorbar(x_ax_non_nan_renal+23,mean_temps_nonnan_renal,yerr=std_temps_nonnan_renal,fmt='o',markersize=5,capsize=5,color='C1',label='Renal')

print('Healthy Kidney, STD per slice',std_temps_nonnan_renal)
ax.legend()




handles, labels = ax.get_legend_handles_labels()
unique = dict()
for handle, label in zip(handles, labels):
    if label not in unique:
        unique[label] = handle
ax.legend(unique.values(), unique.keys(),ncol=1,prop={'size':9})

ax.set_xlabel('Dataset')
ax.set_yticks([25,30,35,40])
ax.set_ylabel(r'T$[^\circ C]$')



# Final

In [ ]:
def fit_aucr_vs_temp(ax,aucr,temp,color):
    def lin_fun(x,M,k):
        return x*k+M
    coeff,pcov = curve_fit(lin_fun,aucr,temp)
    residuals = temp - lin_fun(aucr, *coeff)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((temp-np.mean(temp))**2)
    r_squared = 1 - (ss_res / ss_tot)
    print('intercept,slope:',coeff,'dslope,dintercept',np.sqrt(np.diag(pcov)))
    print('R2',r_squared)
    import seaborn as sns
    sns.regplot(x=aucr,y=temp,ci=90,ax=ax,scatter=False,color=color)
    

In [ ]:
plt.close('all')
fig, ax = plt.subplots(2,1, figsize=(3.42,7),tight_layout=True)

add_points=True

fit_aucr_vs_temp(ax[0],dataframe_GBM['AUCR'],dataframe_GBM['T'],color='C0')
ax[0].errorbar(no_tumor_slice_df_gbm['AUCR'],no_tumor_slice_df_gbm['T'],color='k',yerr=no_tumor_slice_df_gbm['dT'],xerr=no_tumor_slice_df_gbm['dAUCR'],capsize=5,fmt='o',label='No tumor')
ax[0].errorbar(tumor_slice_df_gbm['AUCR'],tumor_slice_df_gbm['T'],color='r',yerr=tumor_slice_df_gbm['dT'],xerr=tumor_slice_df_gbm['dAUCR'],capsize=5,fmt='o',label='tumor')
ax[0].legend()


ax[0].set_xlabel('AUCR')
ax[0].set_ylabel('T [$^\circ$ C]')




# Plot the second subplot with boxplot
healthy_pat_raveld=np.ravel(np.array(healthy_pat_mean_csi))
healthy_pat_raveld = [x for x in healthy_pat_raveld if not math.isnan(x)]
ax[1].boxplot(healthy_pat_raveld, positions=[1], widths=0.5, medianprops=dict(color="red"))

ax[1].boxplot(mean_per_patient_utsw, positions=[2], widths=0.5, medianprops=dict(color="red"))

ax[1].boxplot(mean_temps_nonnan_renal,positions=[3],widths=0.5,medianprops=dict(color="red"))

# Plot the boxplot for black points
ax[1].boxplot([no_tumor_slice_df_gbm['T']], positions=[4], widths=0.5,  medianprops=dict(color="red"))

ax[1].boxplot([tumor_slice_df_gbm['T']], positions=[5], widths=0.5,  medianprops=dict(color="red"))


if add_points == True:
    x_positions = np.ones(len(healthy_pat_raveld))  # All points align with the boxplot's x position
    ax[1].scatter(x_positions, healthy_pat_raveld,  facecolors='none',ec='k',zorder=2)
    x_positions = np.ones(len(mean_per_patient_utsw))+1  # All points align with the boxplot's x position
    ax[1].scatter(x_positions, mean_per_patient_utsw, facecolors='none',ec='k', zorder=2)
    x_positions = np.ones(len(mean_temps_nonnan_renal))+2  # All points align with the boxplot's x position
    ax[1].scatter(x_positions, mean_temps_nonnan_renal, facecolors='none',ec='k', zorder=2)
    
    x_positions = np.ones(len(no_tumor_slice_df_gbm['T']))+3  # All points align with the boxplot's x position
    ax[1].scatter(x_positions, no_tumor_slice_df_gbm['T'], facecolors='none',ec='k', zorder=2)
    x_positions = np.ones(len(tumor_slice_df_gbm['T']))+4  # All points align with the boxplot's x position
    ax[1].scatter(x_positions, tumor_slice_df_gbm['T'], facecolors='none',ec='k', zorder=2)


ax[1].set_xticks(range(1,6), [str(key) for key in ['Healthy CSI','Healthy SliceSpec','Healthy Kidney','GBM (no tumor)','GBM']],rotation=90)
ax[1].set_ylim([24,42])
ax[0].set_ylim([24,42])

ax[1].set_yticks([25,30,35,40])
ax[1].set_ylabel('T[$^\circ$C]')


print('GBM: AUCR vs Temp:',scipy.stats.ttest_ind(dataframe_GBM['AUCR'],dataframe_GBM['T']))
print('GBM tumor vs non tumor GBM slices',scipy.stats.ttest_ind(no_tumor_slice_df_gbm['T'],tumor_slice_df_gbm['T']).pvalue.round(3))
print('Slice spec kidney vs GBM all slices',scipy.stats.ttest_ind(mean_temps_nonnan_renal,dataframe_GBM['T']).pvalue.round(3))
print('Slice spec brain vs GBM all slices',scipy.stats.ttest_ind(mean_per_patient_utsw,dataframe_GBM['T']).pvalue.round(3))
print('CSI brain, GBM all slices',scipy.stats.ttest_ind(healthy_pat_raveld,dataframe_GBM['T']).pvalue.round(3))

print('------')
print('Slice spec kidney vs GBM tumor slices',scipy.stats.ttest_ind(mean_temps_nonnan_renal,tumor_slice_df_gbm['T']).pvalue.round(3))
print('Slice spec brain vs GBM tumor slices',scipy.stats.ttest_ind(mean_per_patient_utsw,tumor_slice_df_gbm['T']).pvalue.round(3))
print('CSI brain, GBM tumor slices',scipy.stats.ttest_ind(healthy_pat_raveld,tumor_slice_df_gbm['T']).pvalue.round(3))


print('------')
print('Slice spec kidney vs GBM non tumor slices',scipy.stats.ttest_ind(mean_temps_nonnan_renal,no_tumor_slice_df_gbm['T']).pvalue.round(3))
print('Slice spec brain vs GBM non tumor slices',scipy.stats.ttest_ind(mean_per_patient_utsw,no_tumor_slice_df_gbm['T']).pvalue.round(3))
print('CSI brain, GBM non tumor slices',scipy.stats.ttest_ind(healthy_pat_raveld,no_tumor_slice_df_gbm['T']).pvalue.round(3))





In [ ]:
print('Healthy slice spec',np.round(np.mean(mean_per_patient_utsw),1),'±',np.round(np.std(mean_per_patient_utsw),1))
print('GBM',np.round(np.nanmean(dataframe_GBM['T']),1),'±',np.round(np.nanstd(dataframe_GBM['T']),1))
print('Healthy CSI',np.round(np.nanmean(healthy_pat_raveld),1),'±',np.round(np.nanstd(healthy_pat_raveld),1))
print('Healthy Kidney',np.round(np.nanmean(mean_temps_nonnan_renal),1),'±',np.round(np.nanstd(mean_temps_nonnan_renal),1))